# Lecture x: Response API vs Chat Completions

## Topics to cover:
1. Why use Response API over Chat Completions
2. Streaming Responses
3. Prompt Engineering
4. Function Calling
5. A Simple Agent

It is a never-ending battle of being up-to-date with the latest changes in AI. There will always be changes to APIs and their documentation, new tools, deprecations of previous API endpoints and tools.

Our job as AI Engineers is to make sure we are up-to-date with the relevant documentation of the tools we are using. Usually, providers will give ample notice before any major changes are introduced and old methods are depreciated.

## Why the Responses API
As mentioned in OpenAIs documentation for [Responses vs Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

- Better for agentic workflows
- Stateful API
- Model availability
- Built-in tool use (Like OpenAIs own 'file search' and 'computer use')

## What the basic Chat Completions looked like

In [ ]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4.1",
  messages=[
    {"role": "developer", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)
